# Interagir avec le terminal Xterm

---

> Dans ce dernier chapitre d'introduction, nous allons importer le paquet xterm présenté ici [http://xtermjs.org/](http://xtermjs.org/) afin de pouvoir obtenir davantage d'interactions entre le code web assembly et le site web.

![XTERM](pictures/xterm_prez.png)

## Installation du paquet xterm avec npm

- **Installer le nouveau paquet :**

`npm install xterm`
```
added 1 package, and audited 312 packages in 3s

39 packages are looking for funding
  run `npm fund` for details

found 0 vulnerabilities

```

- Le fichier **package.json** contient maintenant la dépendance xterm :
```
{
    "scripts": {
        "serve": "webpack-dev-server"
    },
    "dependencies": {
        "first-step": "^0.1.0",
        "xterm": "^5.1.0"
    },
    "devDependencies": {
        "webpack": "^5.82.1",
        "webpack-cli": "^5.1.1",
        "webpack-dev-server": "^4.15.0"
    }
}

```


## Intégration a notre projet

*Je passe très vite sur cette partie et je fais en plus simple. En tant que dev web, j'imagine que vous comprenez bien ce que je fais ici.*

### Récupération du fichier css

> On utilisera la feuille de style du paquet xterm :

- À partir du sous-dossier `public/` :

`mkdir css`

`cd css`

`ln -s ../../node_modules/xterm/css/xterm.css xterm.css`

- Ajout de la feuille au `head` du fichier `index.html` :
```
        <link rel="stylesheet" href="css/xterm.css" />
```

### Ajout de l'élément terminal au DOM

- Ajout de l'élément avec l'id terminal au `body` :

```
        <div id="terminal"></div>
```

### Utilisation de l'exemple

- Reprenons l'exemple de la page xterm et ajoutons ces lignes au début du fichier `index.js` :

```
import { Terminal } from 'xterm';
const terminal = new Terminal();
terminal.open(document.getElementById('terminal'));
terminal.write('Hello from \x1B[1;3;31mxterm.js\x1B[0m $ ');
```

- Après régénération, la page Web devra ressembler  à cela :

![XTERM2](pictures/xterm_first_page.png)

## Communication avec notre module

> Le but ici sera de pouvoir interagir avec une fonction  ecrite en rust et nommee pour l'exemple `eval_key` pour chacune des touche pressees, la fonction recevra la `event.key` du listener `keydown` et retournera un ou plusieurs caracteres a afficher dans le terminal xterm.

```
    document.addEventListener('keydown', (event) => {
        console.log(event.key);
        var result = mod.eval_key(event.key);
        terminal.write(result);
    });
```

**Voici ce a quoi devrait ressembler le fichier index.js.**

```
import { Terminal } from 'xterm';
const terminal = new Terminal();
terminal.open(document.getElementById('terminal'));
terminal.write('Hello from \x1B[1;3;31mxterm.js\x1B[0m $ ');

import("./node_modules/first-step/first_step.js").then((mod) => {
    mod.greet("WebAssembly with npm");
    document.addEventListener('keydown', (event) => {
        console.log(event.key);
        // May use event.keyCode instead 
        var result = mod.eval_key(event.key);
        terminal.write(result);
    });
});

```
*Il ne manque plus qu'à définir la fonction eval_key cote Rust.*

## L'implémentation minimale

- La fonction devant interagir avec le code javascript, elle sera préfixée par la macro `#[wasm_bindgen]` qui se chargera de la rendre accessible depuis l'extérieur et de gérer la conversion des types entre JS et rust.

```
#[wasm_bindgen]
pub fn eval_key(key_value: &str) -> String {
    key_value.to_string() // Transforme la chaine en String allouee dynamiquement
}
```

**On retournera ici la chaine de caractere a l'identique !**

![XTERM3](pictures/xterm_second_page.png)

*L'inconvenient de cette fonction c'est que les sequences speciales ne sont pas gerees, plutut que d'affichier Enter, nous devrions retourner a la ligne `\n\r`.*

## Aller plus loin...

Plusieurs améliorations sont possibles ici :

> - Vous pouvez modifier le code de façon a ce que le **retour à la ligne** soit correctement géré.
> - Trouver un moyen de **stocker les inputs utilisateurs** et retournez quelque chose lors de l'appui de la touche entrée. *Vous aurez peut-être besoin d'une variable globale.*
> - Si vous avez utilisé une variable globale pour l'exercice précédent, consulter le chapitre **2.18.2.1. constructor** et essayez d'en conclure quelque chose d'utile pour votre code.

**DANS CETTE PARTIE, NOUS AVONS DÉCOUVERT COMMENT APPELER UNE FONCTION ÉCRITE EN RUST WEBASSEMBLY DEPUIS UN MODULE JAVASCRIPT. DANS LES PROCHAINS CHAPITRES, NOUS VERRONS COMMENT DÉMARRER L'EXÉCUTION DEPUIS LE MODULE WASM, DE GÉRER LE DOM ET D'IMPORTER LES AUTRES MODULES.**